# XML



Тип данных <b>```xml```</b>.


In [1]:
declare @xml xml
set @xml = '<root/>'
select @xml
go

set nocount on

declare @xml xml
set @xml = '<root> </root>'
select @xml
go

declare @xml xml
set @xml = 'Text'
select @xml
go

declare @xml xml
set @xml = '<root><element Attr="Text" /></root>'
select @xml
go

declare @xml xml
set @xml = '<root><element Attr=''Text'' /></root>'
select @xml
go

declare @xml xml
set @xml = '
<root>
	<element Attr=''Text1'' />
	<element Attr=''Text2''>
		<subelement>
			Text3
		</subelement>
	</element>
</root>
'
select @xml
go

(1 row affected)

Total execution time: 00:00:00.0062984

Commands completed successfully.

Total execution time: 00:00:00.0037364

Commands completed successfully.

...

(No column name)
<root />


(No column name)
Text


(No column name)
"<root><element Attr=""Text"" /></root>"


(No column name)
"<root><element Attr=""Text"" /></root>"


(No column name)
"<root><element Attr=""Text1"" /><element Attr=""Text2""><subelement> Text3 </subelement></element></root>"


Метод query() (XPath как подмножество XQuery):


In [3]:
declare @xml xml
set @xml = '
<root>
	<element Attr=''Text1'' />
	<element Attr=''Text2''>
		<subelement>
			Text3
		</subelement>
	</element>
</root>
'
select @xml.query('//root/element[@Attr="Text1"]') with_attr1, 
       @xml.query('//element/subelement')          with_subelement,
       @xml.query('//element/subelement/text()')   subelement_text
go

(1 row affected)

Total execution time: 00:00:00.0233780

with_attr1,with_subelement,subelement_text
"<element Attr=""Text1"" />",<subelement> Text3 </subelement>,Text3


Метод exist():

In [6]:
declare @xml xml
set @xml = '
<root>
	<element Attr=''Text1'' />
	<element Attr=''Text2''>
		<subelement>
			Text3
		</subelement>
	</element>
</root>
'
select @xml, @xml.exist('subelement'), @xml.exist('//root/*/subelement'), @xml.exist('//root/element[@Attr]')
go


(1 row affected)

Total execution time: 00:00:00.0047927

(No column name),(No column name),(No column name),(No column name)
"<root><element Attr=""Text1"" /><element Attr=""Text2""><subelement> Text3 </subelement></element></root>",0,1,1


Метод value():

In [0]:
declare @xml xml
set @xml = '
<root>
	<element Attr=''Text1'' />
	<element Attr=''Text2''>
		<subelement>
			Text3
		</subelement>
	</element>
</root>
'

select @xml.value('(//competitions/row/result/athlete_name)[1]','nvarchar(100)') 

select cast(@xml.query('(//competitions/row/result/athlete_name)[1]/text()') as nvarchar(100))


Загрузка данных (требуются права на выполнение openrowset):

In [6]:
declare @xml    xml

select @xml = BulkColumn    
from openrowset (bulk N'c:/temp/SwimmingCompetition.xml', single_blob)  q


select cast(@xml.query('(//competitions/row/result/athlete_name)[1]/text()') as nvarchar(100))

select @xml.value('(//competitions/row/result/athlete_name)[1]','nvarchar(100)') 
go

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.4238847

(No column name)
САФОНОВА Анастасия


(No column name)
САФОНОВА Анастасия


Использование FLOWR для трансформации XML документа (еще одно подмножество языка XQuery):

In [0]:
declare @xml xml
set @xml = (select * from openrowset (bulk 'samples/SwimmingCompetition.xml', single_blob)  q)

select @xml.query(
'
<doc>
{
	for $Person in /root/Person
	where $Person/@City = "Seattle"
	return
	<Person FirstName="{$Person/Name/FirstName}">
	<City>{ $Person/@City }</City>
	<Address>{ $Person/@Address }</Address>
	</Person>
}
</doc>
'
)
go

Тот же код, но используя внешний параметр/переменную: